In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hemul import heaan
import hemul.HEAAN as he
from muxcnn.resnet_HEAAN import ResNetHEAAN
from muxcnn.utils import load_params, load_img
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv
from muxcnn.utils import get_channel_last, get_conv_params
from muxcnn.hecnn_par import MultParPack
from muxcnn.hecnn_par import forward_convbn_par

## Torch model

In [3]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

img_tensor = load_img("./cute.jpg", hi=32, wi=32)

## Image

In [4]:
imgl = get_channel_last(img_tensor[0].detach().numpy())
ki = 1 # initial ki
hi, wi, ch = imgl.shape

# FHE

In [6]:
rot_l = [2**i for i in range(15)]

# prepare for all rotations
rot_l = rot_l + [2**15-1, 
         2**15-33, 2**15-32, 2**15-31,
         2**15-17, 2**15-16, 2**15-15, 
         2**15-9,2**15-8, 2**15-7] + [3,5,7,9,15,17, 31, 33]

# try 40 & 800
hec = heaan.HEAANContext(15, 30, 600, boot=False, load_sk=True, rot_l=rot_l)

Initializing the scheme as the data owner
Loading a secret key from:  ./SecretKey.txt
loading secret key done.
HEAAN CKKS setup is ready 


In [7]:
fhemodel = ResNetHEAAN(model, hec)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


In [8]:
ctxt = fhemodel.pack_img_ctxt(img_tensor)

# forward early

In [9]:
ctxt0, outs0 = fhemodel.forward_early(ctxt, 1, 32, 32)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
Check 1
[ 1.23082668  0.48696742  0.27533217 ... -0.6505671  -0.61289941
 -0.4793413 ]
APPR 30 540
[ 1.23082668  0.48696742  0.27533217 ... -0.6505671  -0.61289941
 -0.4793413 ]
min max -5.745795823261249 5.543789504823771
APPR 30 390
[ 0.60359263  0.25015599  0.14227105 ... -0.33194343 -0.31326628
 -0.24630497]
min max -1.1776196965637782 1.1776222061654924
APPR 30 240
[ 1.04730287  1.14598853  0.96842739 ... -1.03412835 -1.06764562
 -1.14749952]
min max -1.153626775194164 1.1530091993361065
APPR 30 90
[ 0.99703646  1.00711951  1.00434685 ... -0.99833077 -0.99489298
 -1.00535495]
min max -1.0114610301698828 1.0114035415357514
bootstrap done
AFTER bootstrap 30 600


In [10]:
dec = hec.decrypt(ctxt0)
print(dec[::1000])

[ 1.23080732e+00 -2.19172272e-06  9.00840631e-01  4.97676851e-07
  1.03884788e+00 -1.64028233e-03  1.72917598e-02  2.26551253e-01
  5.20674528e-06 -1.52607481e-06  6.20172372e-07 -2.70625058e-06
 -1.84631419e-06  4.46615700e-01 -3.47298700e-07  1.06025746e+00
  1.89328142e-05  4.77775358e-08  2.69853589e-01  3.97405181e+00
  1.03376779e+00 -5.89558119e-06  5.28603341e-01  2.52202193e+00
 -2.34386009e-06  8.40872827e-02 -1.59815229e-05  3.08667737e-05
  7.58121725e-06  6.15326402e-01  5.37449316e-01  4.76051826e-01
  2.18506417e-06]


In [ ]:
ctxt1, outs1 = fhemodel.forward_bb(model.layer1[0], ctxt0, outs0)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8


In [ ]:
ctxt2, outs2 = fhemodel.forward_bb(model.layer2[0], ctxt1, outs1)

In [ ]:
ctxt3, outs3 = fhemodel.forward_bb(model.layer3[0], ctxt2, outs2)

In [5]:
# early conv and bn
_, ins0, outs0 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins0)

In [6]:
# Muxed
U, ins1, outs1 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins1)

out1, un1 = forward_convbn_par(model.conv1, 
                              model.bn1, ct_a, ins1)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
2.7468 s


## Activation